# import

In [1]:
import cv2
import os
import os.path as osp
from glob import glob
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from util import load_json, save_json
from copy import copy
import shutil

In [2]:
import time

# path

In [3]:
data_name = 'player'
input_dir = osp.join('../input', data_name)
data_dir = osp.join('../output', data_name, 'tracking')
label_save_dir = osp.join('../output', data_name, 'label')
reid_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')
# movie_dir = osp.join('../output', data_name, 'movie')
movie_dir = osp.join('../output', data_name, 'movie_process')
mot_pid_dir = osp.join('../output', data_name, 'mot_pid')
mot_process_save_dir = osp.join('../output', data_name, 'mot_pid_process')

In [4]:
data_names = ['player', 'court', 'ball']

# function

In [5]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [6]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

In [7]:
def draw_bbox(img, bboxes, c=(0,255,0)):
    for bbox in bboxes:
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), 
                      (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), 
                      (0,255,0), 2, lineType=cv2.LINE_AA)
        ct = [bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2]
        txt = '{}'.format(bbox[4])
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    (0, 0, 0), thickness=3, lineType=cv2.LINE_AA)
        cv2.putText(img, txt, (int(ct[0]), int(ct[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                    c, thickness=2, lineType=cv2.LINE_AA)

# all

In [9]:
games = ['game1']
# games = None

clips = ['Clip1']
# clips = None

In [69]:
if games is None:
    games = listdirs(input_dir, 'game')
    
columns = ['frame_index', 'tracking_id', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'image_name']
for game in tqdm(games):
    game_dir = osp.join(input_dir, game)
    if clips is None:
        clips = listdirs(game_dir, 'Clip')
    for clip in clips:
        clip_dir = osp.join(game_dir, clip)
        file_names = listdirs(clip_dir)
        image_names = [file_name for file_name in file_names if osp.splitext(file_name)[1]=='.jpg']
        label_names = [file_name for file_name in file_names if osp.splitext(file_name)[1]=='.json']
        
        assert len(image_names) == len(label_names)
        
        for image_name, label_name in zip(image_names, label_names):
                
            image_path = osp.join('../input', 'player', game, clip, image_name)
            
            player_label_path = osp.join('../input', 'player', game, clip, label_name)
            court_label_path = osp.join('../input', 'court', game, clip, label_name)
            ball_label_path = osp.join('../input', 'ball', game, clip, label_name)

            player_label = load_json(player_label_path)
            court_label = load_json(court_label_path)
            ball_label = load_json(ball_label_path)
            
            assert player_label['imagePath'] == court_label['imagePath'] == ball_label['imagePath']
            
            for l in court_label['shapes']:
                pt = l['points']
                pt = [[int(pt[0][0]),int(pt[0][1])]]
                l['points'] = pt
            
            labels = []
            for num, l in enumerate(ball_label['shapes']):
                pt = l['points']
                if np.isnan(l['points'][0][0]):
                    del ball_label['shapes'][num]
                    continue
                pt = [[int(pt[0][0]),int(pt[0][1])]]
                l['points'] = pt
            
            shapes = copy(player_label['shapes'])
            shapes.extend(court_label['shapes'])
            shapes.extend(ball_label['shapes'])
            
            player_label['shapes'] = shapes
            
            label_save_path = osp.join('../input', 'tennis', game, clip, label_name)
            image_save_path = osp.join('../input', 'tennis', game, clip, image_name)
            
            os.makedirs(osp.dirname(label_save_path), exist_ok=True)
                        
            shutil.copy(image_path, image_save_path)
            save_json(player_label, label_save_path)
                        
    clips = None

100%|██████████| 10/10 [00:17<00:00,  1.70s/it]


In [68]:
l

{'fill_color': None,
 'flags': {},
 'label': 'ball',
 'line_color': None,
 'points': [[400, 117]],
 'shape_type': 'polygon',
 'status': 0,
 'visibilliey': 1}

In [66]:
l['points']

[[400, 117]]

In [67]:
l

{'fill_color': None,
 'flags': {},
 'label': 'ball',
 'line_color': None,
 'points': [[400, 117]],
 'shape_type': 'polygon',
 'status': 0,
 'visibilliey': 1}

In [60]:
l['points']

[[nan, nan]]

In [63]:
del l

In [64]:
l

NameError: name 'l' is not defined

In [58]:
np.isnan(l['points'][0][0])

True

In [56]:
if l['points'][0][0] is np.nan:
    pass

In [46]:
for l in court_label['shapes']:
    pt = l['points']
    pt = [[int(pt[0][0]),int(pt[0][1])]]
    l['points'] = pt

In [48]:
court_label

{'fillColor': [255, 0, 0, 128],
 'flags': {},
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0000.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '0',
   'line_color': None,
   'points': [[643, 297]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '1',
   'line_color': None,
   'points': [[803, 163]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '2',
   'line_color': None,
   'points': [[484, 164]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '3',
   'line_color': None,
   'points': [[328, 570]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '4',
   'line_color': None,
   'points': [[960, 569]],
   'shape_type': 'polygon'},
  {'flags': {},
   'group_id': None,
   'lab

In [43]:
pt

[[1069, 570]]

In [41]:
l['points']

[[1069.6666666666667, 570.0]]

In [40]:
pt

[1069, 570]

In [37]:
pt

[[643.0, 297.5]]

In [35]:
pts

[[1069.6666666666667, 570.0]]

In [34]:
pt

[1069, 570]

In [1]:
label

NameError: name 'label' is not defined

In [73]:
image_save_path

'../input/tennis/game1/Clip11/0000.jpg'

In [71]:
image_path

'../input/ball/game9/Clip9/0222.jpg'

In [67]:
player_label

{'fillColor': [255, 0, 0, 128],
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': 'person',
   'line_color': None,
   'person_name': 'Sharapova',
   'points': [[723, 455], [788, 637]],
   'shape_type': 'rectangle'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': 'person',
   'line_color': None,
   'person_name': 'Halep',
   'points': [[494, 54], [530, 108]],
   'shape_type': 'rectangle'}],
 'version': '4.2.10'}

In [66]:
player_label['shapes'] = shapes

{'fillColor': [255, 0, 0, 128],
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': 'person',
   'line_color': None,
   'person_name': 'Sharapova',
   'points': [[723, 455], [788, 637]],
   'shape_type': 'rectangle'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': 'person',
   'line_color': None,
   'person_name': 'Halep',
   'points': [[494, 54], [530, 108]],
   'shape_type': 'rectangle'}],
 'version': '4.2.10'}

In [60]:
print(player_label['imagePath'])

0001.jpg


In [61]:
print(player_label['imagePath'])
print(court_label['imagePath'])
print(ball_label['imagePath'])

0001.jpg
0000.jpg
0000.jpg


In [43]:
court_label

{'fillColor': [255, 0, 0, 128],
 'flags': {},
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '0',
   'line_color': None,
   'points': [[643.0, 297.5]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '1',
   'line_color': None,
   'points': [[803.0, 163.33333333333337]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '2',
   'line_color': None,
   'points': [[484.6666666666667, 164.16666666666669]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '3',
   'line_color': None,
   'points': [[328.0, 570.0]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '4',
   'line_color': None,
   'points': [[960.5, 569.1666666666667]],
   

In [40]:
shapes = copy(player_label['shapes'])
shapes.extend(court_label['shapes'])
shapes.extend(ball_label['shapes'])

In [41]:
shapes

[{'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': 'person',
  'line_color': None,
  'person_name': 'Sharapova',
  'points': [[723, 455], [788, 637]],
  'shape_type': 'rectangle'},
 {'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': 'person',
  'line_color': None,
  'person_name': 'Halep',
  'points': [[494, 54], [530, 108]],
  'shape_type': 'rectangle'},
 {'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': '0',
  'line_color': None,
  'points': [[643.0, 297.5]],
  'shape_type': 'polygon'},
 {'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': '1',
  'line_color': None,
  'points': [[803.0, 163.33333333333337]],
  'shape_type': 'polygon'},
 {'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': '2',
  'line_color': None,
  'points': [[484.6666666666667, 164.16666666666669]],
  'shape_type': 'polygon'},
 {'fill_color': None,
  'flags': {},
  'group_id': None,
  'label': '3',
  'line_color': None,
  'points': [[328

In [34]:
court_label

{'fillColor': [255, 0, 0, 128],
 'flags': {},
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '0',
   'line_color': None,
   'points': [[643.0, 297.5]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '1',
   'line_color': None,
   'points': [[803.0, 163.33333333333337]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '2',
   'line_color': None,
   'points': [[484.6666666666667, 164.16666666666669]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '3',
   'line_color': None,
   'points': [[328.0, 570.0]],
   'shape_type': 'polygon'},
  {'fill_color': None,
   'flags': {},
   'group_id': None,
   'label': '4',
   'line_color': None,
   'points': [[960.5, 569.1666666666667]],
   

In [35]:
ball_label

{'fillColor': [255, 0, 0, 128],
 'flags': {},
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'label': 'ball',
   'line_color': None,
   'points': [[400, 117]],
   'shape_type': 'polygon',
   'status': 0,
   'visibilliey': 1}],
 'version': '3.21.1'}

In [23]:
label

{'fillColor': [255, 0, 0, 128],
 'flags': {},
 'imageData': None,
 'imageHeight': 720,
 'imagePath': '0222.jpg',
 'imageWidth': 1280,
 'lineColor': [0, 255, 0, 128],
 'shapes': [{'fill_color': None,
   'flags': {},
   'label': 'ball',
   'line_color': None,
   'points': [[400, 117]],
   'shape_type': 'polygon',
   'status': 0,
   'visibilliey': 1}],
 'version': '3.21.1'}

In [11]:
image_names

['0000.jpg',
 '0001.jpg',
 '0002.jpg',
 '0003.jpg',
 '0004.jpg',
 '0005.jpg',
 '0006.jpg',
 '0007.jpg',
 '0008.jpg',
 '0009.jpg',
 '0010.jpg',
 '0011.jpg',
 '0012.jpg',
 '0013.jpg',
 '0014.jpg',
 '0015.jpg',
 '0016.jpg',
 '0017.jpg',
 '0018.jpg',
 '0019.jpg',
 '0020.jpg',
 '0021.jpg',
 '0022.jpg',
 '0023.jpg',
 '0024.jpg',
 '0025.jpg',
 '0026.jpg',
 '0027.jpg',
 '0028.jpg',
 '0029.jpg',
 '0030.jpg',
 '0031.jpg',
 '0032.jpg',
 '0033.jpg',
 '0034.jpg',
 '0035.jpg',
 '0036.jpg',
 '0037.jpg',
 '0038.jpg',
 '0039.jpg',
 '0040.jpg',
 '0041.jpg',
 '0042.jpg',
 '0043.jpg',
 '0044.jpg',
 '0045.jpg',
 '0046.jpg',
 '0047.jpg',
 '0048.jpg',
 '0049.jpg',
 '0050.jpg',
 '0051.jpg',
 '0052.jpg',
 '0053.jpg',
 '0054.jpg',
 '0055.jpg',
 '0056.jpg',
 '0057.jpg',
 '0058.jpg',
 '0059.jpg',
 '0060.jpg',
 '0061.jpg',
 '0062.jpg',
 '0063.jpg',
 '0064.jpg',
 '0065.jpg',
 '0066.jpg',
 '0067.jpg',
 '0068.jpg',
 '0069.jpg',
 '0070.jpg',
 '0071.jpg',
 '0072.jpg',
 '0073.jpg',
 '0074.jpg',
 '0075.jpg',
 '0076.jpg',

In [12]:
label_names

['0000.json',
 '0001.json',
 '0002.json',
 '0003.json',
 '0004.json',
 '0005.json',
 '0006.json',
 '0007.json',
 '0008.json',
 '0009.json',
 '0010.json',
 '0011.json',
 '0012.json',
 '0013.json',
 '0014.json',
 '0015.json',
 '0016.json',
 '0017.json',
 '0018.json',
 '0019.json',
 '0020.json',
 '0021.json',
 '0022.json',
 '0023.json',
 '0024.json',
 '0025.json',
 '0026.json',
 '0027.json',
 '0028.json',
 '0029.json',
 '0030.json',
 '0031.json',
 '0032.json',
 '0033.json',
 '0034.json',
 '0035.json',
 '0036.json',
 '0037.json',
 '0038.json',
 '0039.json',
 '0040.json',
 '0041.json',
 '0042.json',
 '0043.json',
 '0044.json',
 '0045.json',
 '0046.json',
 '0047.json',
 '0048.json',
 '0049.json',
 '0050.json',
 '0051.json',
 '0052.json',
 '0053.json',
 '0054.json',
 '0055.json',
 '0056.json',
 '0057.json',
 '0058.json',
 '0059.json',
 '0060.json',
 '0061.json',
 '0062.json',
 '0063.json',
 '0064.json',
 '0065.json',
 '0066.json',
 '0067.json',
 '0068.json',
 '0069.json',
 '0070.json',
 '0071

In [9]:
file_names

['0000.jpg',
 '0000.json',
 '0001.jpg',
 '0001.json',
 '0002.jpg',
 '0002.json',
 '0003.jpg',
 '0003.json',
 '0004.jpg',
 '0004.json',
 '0005.jpg',
 '0005.json',
 '0006.jpg',
 '0006.json',
 '0007.jpg',
 '0007.json',
 '0008.jpg',
 '0008.json',
 '0009.jpg',
 '0009.json',
 '0010.jpg',
 '0010.json',
 '0011.jpg',
 '0011.json',
 '0012.jpg',
 '0012.json',
 '0013.jpg',
 '0013.json',
 '0014.jpg',
 '0014.json',
 '0015.jpg',
 '0015.json',
 '0016.jpg',
 '0016.json',
 '0017.jpg',
 '0017.json',
 '0018.jpg',
 '0018.json',
 '0019.jpg',
 '0019.json',
 '0020.jpg',
 '0020.json',
 '0021.jpg',
 '0021.json',
 '0022.jpg',
 '0022.json',
 '0023.jpg',
 '0023.json',
 '0024.jpg',
 '0024.json',
 '0025.jpg',
 '0025.json',
 '0026.jpg',
 '0026.json',
 '0027.jpg',
 '0027.json',
 '0028.jpg',
 '0028.json',
 '0029.jpg',
 '0029.json',
 '0030.jpg',
 '0030.json',
 '0031.jpg',
 '0031.json',
 '0032.jpg',
 '0032.json',
 '0033.jpg',
 '0033.json',
 '0034.jpg',
 '0034.json',
 '0035.jpg',
 '0035.json',
 '0036.jpg',
 '0036.json',
 